In [2]:
import pandas as pd

from tqdm import tqdm

In [5]:
stream_dist = pd.read_csv("LamaH-CE/raw/B_basins_intermediate_all/1_attributes/Stream_dist.csv", sep=";")

In [37]:
def check_reliable(gauge_id):
    ts = pd.read_csv(f"LamaH-CE/raw/D_gauges/2_timeseries/hourly/ID_{gauge_id}.csv",
                         sep=";", usecols=["YYYY", "MM", "DD", "hh", "mm", "qobs"])
    if (ts["qobs"] >= 0).all():
        sub_ts = ts[(ts["YYYY"] >= 2000) & (ts["YYYY"] <= 2017)]
        if len(sub_ts) == (18 * 365 + 5) * 24:  # number of hours in 2000-2017
            return True
    return False

def discover_reliable_gauges(gauge_id):
    if not check_reliable(gauge_id):
        return {}
    
    predecessors = set(stream_dist[stream_dist["NEXTDOWNID"] == gauge_id]["ID"])
    return {gauge_id}.union(*(discover_reliable_gauges(pred) for pred in predecessors))

In [50]:
results = {}
for i in tqdm(range(1, 883)):
    results[i] = discover_reliable_gauges(i)

100%|█████████████████████████████████████████████████████████████████████████████████| 882/882 [04:50<00:00,  3.03it/s]


In [56]:
max(map(len, results.values()))

59